In [1]:
import sys
sys.path.insert(0, "/cndd/fangming/CEMBA/snmcseq_dev")

from multiprocessing import Pool,cpu_count
from functools import partial

from scipy import sparse

from __init__ import *
from __init__jupyterlab import *
import snmcseq_utils

import pickle

In [11]:
data_dir = '/cndd2/fangming/projects/scf_enhancers/enhancer_cell_level/mcg/'
ti = time.time()

cols = ['mCG', 'CG', 'mCH', 'CH']
with snmcseq_utils.cd(data_dir):
    files = np.sort(glob.glob('*.gz'))
    print(len(files))
    
    mats = {col: [] for col in cols}
    for i, file in enumerate(files):
        if i%100 == 0:
            print(i, time.time()-ti)
            
        df = pd.read_csv(file, sep='\t')
        
        if i == 0:
            regions = df[['chr', 'start', 'end']].copy()
#             break

        for col in cols:
            mats[col].append(sparse.coo_matrix(df[col].values)
                            )
    
    mats = {col: sparse.vstack(mats[col]) for col in mats.keys()}
        
        
#         df['cg']
#         print(df.shape)
#         df.head()

    

9931
0 0.058374881744384766


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ValueError: blocks must be 2-D

In [3]:
for col, mat in mats.items():
    print(col, mat.shape)

mCG (9931, 233514)
CG (9931, 233514)
mCH (9931, 233514)
CH (9931, 233514)


In [4]:
output_file = os.path.join(data_dir, 'combined_enh_cell_mc_mats.pkl')

with open(output_file, 'wb') as handle:
    pickle.dump(mats, handle)
    
with open(output_file, 'rb') as handle:
    mats_from_file = pickle.load(handle)
    

In [5]:
for col, mat in mats_from_file.items():
    print(col, mat.shape)

mCG (9931, 233514)
CG (9931, 233514)
mCH (9931, 233514)
CH (9931, 233514)


In [8]:
cell_ids = [file[len('enhancer_'):-len('.tsv.gz')] for file in files]
output = os.path.join(data_dir, "combined_enh_cell_mc_mats.cell")
snmcseq_utils.export_single_textcol(output, cell_ids)
cell_ids[:3]

['171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_1_CEMBA171206_3C_3_A10_AD001_indexed',
 '171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_1_CEMBA171206_3C_3_A10_AD002_indexed',
 '171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_1_CEMBA171206_3C_3_A10_AD004_indexed']

In [13]:
output = os.path.join(data_dir, "combined_enh_cell_mc_mats.region")
regions.to_csv(output, header=True, index=False, sep='\t')